# Combining Targets 4 and 7

The purpose of this notebook is to look at how our models accuracy looks when we mix targets 4 and 7 together and it takes a look at the other features sets gain/lose importance across the data